In [1]:
basketball = 0
designer = 0
kids = 0
vegan = 0
party = 0
la_lat_lon = [34.0522,-118.2437]
madrid_lat_lon = [40.4168,-3.7038]
lond_lat_lon = [51.5074,-0.1278]


In [2]:
from dotenv import load_dotenv
import src.funcioncitas as sr
import os
import requests
from pandas import json_normalize
import json
from functools import reduce
import operator
import geopandas as gpd
from cartoframes.viz import Map, Layer, popup_element
import pandas as pd
from pymongo import MongoClient
from pymongo import GEOSPHERE
import folium 

In [4]:
load_dotenv()

True

In [5]:
client_id = os.getenv("token1")
client_secret = os.getenv("token2")

In [6]:
url_query = 'https://api.foursquare.com/v2/venues/explore'

In [7]:
parametros = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{la_lat_lon[0]}, {la_lat_lon[1]}",
    "query": "Starbucks",
    "limit": 200,
    "radius": 10000
}

In [8]:
resp = requests.get(url_query, params=parametros).json()

In [9]:
resp

{'meta': {'code': 200, 'requestId': '6135c05b31f45e4f70627d69'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Los Angeles',
  'headerFullLocation': 'Los Angeles',
  'headerLocationGranularity': 'city',
  'query': 'starbucks',
  'totalResults': 95,
  'suggestedBounds': {'ne': {'lat': 34.14220009000009,
    'lng': -118.13527629608807},
   'sw': {'lat': 33.96219990999991, 'lng': -118.35212370391194}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '53487060498e5e04e7503d6c',
       'name': 'Starbucks',
       'contact': {},
       'location': {'address': '639 N Broadway',
        'crossStreet': 'W.Cesar Chavez Ave',
        'lat': 34.05877990

In [10]:
data= resp["response"]["groups"][0]["items"]

In [55]:
df = json_normalize(data)

In [12]:
mapa_nombre = ["venue", "name"]
mapa_latitud = ["venue", "location", "lat"]
mapa_longitud = ["venue", "location", "lng"]

In [13]:
def getFromDict(diccionario, mapa):
    return reduce(operator.getitem,mapa,diccionario)

In [14]:
def type_point(lista):
    return {"type": "Point", 
            "coordinates": lista }

In [15]:
unjsonnuevo = []
for dic in data:
    paralista= {}
    paralista["nombre"]= getFromDict(dic,mapa_nombre)
    paralista["latitud"]= getFromDict(dic,mapa_latitud)
    paralista["longitud"]= getFromDict(dic,mapa_longitud)
    paralista["location"]=  type_point([paralista["longitud"],paralista["latitud"]])
    unjsonnuevo.append(paralista)

In [16]:
with open('starbucks.json', 'w') as f:
    json.dump(unjsonnuevo, f)

In [17]:
unjsonnuevo

[{'nombre': 'Starbucks',
  'latitud': 34.05877990357595,
  'longitud': -118.24024876785406,
  'location': {'type': 'Point',
   'coordinates': [-118.24024876785406, 34.05877990357595]}},
 {'nombre': 'Starbucks',
  'latitud': 34.049518247264665,
  'longitud': -118.24190789611599,
  'location': {'type': 'Point',
   'coordinates': [-118.24190789611599, 34.049518247264665]}},
 {'nombre': 'Starbucks Reserve',
  'latitud': 34.07718627690432,
  'longitud': -118.26489718236814,
  'location': {'type': 'Point',
   'coordinates': [-118.26489718236814, 34.07718627690432]}},
 {'nombre': 'Starbucks',
  'latitud': 34.04816305210899,
  'longitud': -118.25852634103549,
  'location': {'type': 'Point',
   'coordinates': [-118.25852634103549, 34.04816305210899]}},
 {'nombre': 'Starbucks',
  'latitud': 34.05208,
  'longitud': -118.26327,
  'location': {'type': 'Point', 'coordinates': [-118.26327, 34.05208]}},
 {'nombre': 'Starbucks',
  'latitud': 34.05292207428664,
  'longitud': -118.24789695677013,
  'loca

In [18]:
starbucks = pd.DataFrame(unjsonnuevo)
starbucks

,nombre,latitud,longitud,location
0,Starbucks,34.058780,-118.240249,"{'type': 'Point', 'coordinates': [-118.2402487..."
1,Starbucks,34.049518,-118.241908,"{'type': 'Point', 'coordinates': [-118.2419078..."
2,Starbucks Reserve,34.077186,-118.264897,"{'type': 'Point', 'coordinates': [-118.2648971..."
3,Starbucks,34.048163,-118.258526,"{'type': 'Point', 'coordinates': [-118.2585263..."
4,Starbucks,34.052080,-118.263270,"{'type': 'Point', 'coordinates': [-118.26327, ..."
...,...,...,...,...
90,Starbucks,33.988958,-118.213739,"{'type': 'Point', 'coordinates': [-118.2137394..."
91,Starbucks,33.988826,-118.309897,"{'type': 'Point', 'coordinates': [-118.3098972..."
92,Starbucks,34.084384,-118.327056,"{'type': 'Point', 'coordinates': [-118.3270564..."
93,Starbucks,34.022042,-118.290178,"{'type': 'Point', 'coordinates': [-118.2901779..."


In [19]:
gdf = gpd.GeoDataFrame(starbucks, geometry = gpd.points_from_xy(starbucks.longitud, starbucks.latitud))
gdf

,nombre,latitud,longitud,location,geometry
0,Starbucks,34.058780,-118.240249,"{'type': 'Point', 'coordinates': [-118.2402487...",POINT (-118.24025 34.05878)
1,Starbucks,34.049518,-118.241908,"{'type': 'Point', 'coordinates': [-118.2419078...",POINT (-118.24191 34.04952)
2,Starbucks Reserve,34.077186,-118.264897,"{'type': 'Point', 'coordinates': [-118.2648971...",POINT (-118.26490 34.07719)
3,Starbucks,34.048163,-118.258526,"{'type': 'Point', 'coordinates': [-118.2585263...",POINT (-118.25853 34.04816)
4,Starbucks,34.052080,-118.263270,"{'type': 'Point', 'coordinates': [-118.26327, ...",POINT (-118.26327 34.05208)
...,...,...,...,...,...
90,Starbucks,33.988958,-118.213739,"{'type': 'Point', 'coordinates': [-118.2137394...",POINT (-118.21374 33.98896)
91,Starbucks,33.988826,-118.309897,"{'type': 'Point', 'coordinates': [-118.3098972...",POINT (-118.30990 33.98883)
92,Starbucks,34.084384,-118.327056,"{'type': 'Point', 'coordinates': [-118.3270564...",POINT (-118.32706 34.08438)
93,Starbucks,34.022042,-118.290178,"{'type': 'Point', 'coordinates': [-118.2901779...",POINT (-118.29018 34.02204)


In [16]:
client = MongoClient("localhost:27017")
db = client.get_database("new_office")

In [17]:
LA = db.get_collection("LA")

In [18]:
query = [{
        '$geoNear': {'near': [la_lat_lon[1],la_lat_lon[0]],
            'distanceField': 'distancia',
            'maxDistance': 10000,
            'query': { 'nombre': {'$regex':'Starbucks' }},
            'distanceMultiplier': 6371.008,
            #'includeLocs': "dist.location",
            'spherical': True,
            }
        }]

In [19]:
geoloc = LA.aggregate(query)

In [20]:
test = pd.DataFrame(geoloc)

In [21]:
test

""


In [22]:
test.distancia.min()

AttributeError: 'DataFrame' object has no attribute 'distancia'

In [23]:
parametros2 = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{la_lat_lon[0]}, {la_lat_lon[1]}",
    "query": "Vegan Restaurant",
    "limit": 200,
    "radius": 10000
}

NameError: name 'client_id' is not defined

In [24]:
resp2 = requests.get(url_query, params=parametros2).json()

NameError: name 'url_query' is not defined

In [29]:
data2 = resp2["response"]["groups"][0]["items"]

In [30]:
data2

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '5518423c498edbf62812dbe4',
   'name': 'Cafe Gratitude',
   'contact': {},
   'location': {'address': '300 S Santa Fe Ave Ste A',
    'crossStreet': 'at E 3rd St',
    'lat': 34.04562851412795,
    'lng': -118.23255871190399,
    'labeledLatLngs': [{'label': 'display',
      'lat': 34.04562851412795,
      'lng': -118.23255871190399}],
    'distance': 1261,
    'postalCode': '90013',
    'cc': 'US',
    'neighborhood': 'Arts District',
    'city': 'Los Angeles',
    'state': 'CA',
    'country': 'United States',
    'formattedAddress': ['300 S Santa Fe Ave Ste A (at E 3rd St)',
     'Los Angeles, CA 90013',
     'United States']},
   'categories': [{'id': '4bf58dd8d48988d1d3941735',
     'name': 'Vegetarian / Vegan Restaurant',
     'pluralName': 'Vegetarian / Vegan Restaurants',
     'shortName': 'Vegetarian / Vegan',
   

In [47]:
vegan = []
for dic in data2:
    paralista= {}
    paralista[f"nombre."]= getFromDict(dic,mapa_nombre)
    paralista["latitud"]= getFromDict(dic,mapa_latitud)
    paralista["longitud"]= getFromDict(dic,mapa_longitud)
    paralista["location"]=  type_point([paralista["longitud"],paralista["latitud"]])
    vegan.append(paralista)

In [48]:
vegano = pd.DataFrame(vegan)

In [49]:
vegano

,nombre2,latitud,longitud,location
0,Cafe Gratitude,34.045629,-118.232559,"{'type': 'Point', 'coordinates': [-118.2325587..."
1,Au Lac,34.056475,-118.250488,"{'type': 'Point', 'coordinates': [-118.2504878..."
2,Veggie Grill,34.048655,-118.254717,"{'type': 'Point', 'coordinates': [-118.2547171..."
3,Shojin,34.045226,-118.238560,"{'type': 'Point', 'coordinates': [-118.2385599..."
4,Modern Times Beer: The Dankness Dojo,34.044133,-118.257600,"{'type': 'Point', 'coordinates': [-118.2575999..."
...,...,...,...,...
95,The Vegan Indian,34.091187,-118.328514,"{'type': 'Point', 'coordinates': [-118.3285140..."
96,The Blue Rose,34.094118,-118.327508,"{'type': 'Point', 'coordinates': [-118.327508,..."
97,Brothers Meatballs,34.097830,-118.325810,"{'type': 'Point', 'coordinates': [-118.32581, ..."
98,Golden Bridge Yoga,34.096240,-118.327560,"{'type': 'Point', 'coordinates': [-118.3275604..."


In [50]:
gdf2 = gpd.GeoDataFrame(vegano, geometry = gpd.points_from_xy(vegano.longitud, vegano.latitud))
gdf2

,nombre2,latitud,longitud,location,geometry
0,Cafe Gratitude,34.045629,-118.232559,"{'type': 'Point', 'coordinates': [-118.2325587...",POINT (-118.23256 34.04563)
1,Au Lac,34.056475,-118.250488,"{'type': 'Point', 'coordinates': [-118.2504878...",POINT (-118.25049 34.05647)
2,Veggie Grill,34.048655,-118.254717,"{'type': 'Point', 'coordinates': [-118.2547171...",POINT (-118.25472 34.04865)
3,Shojin,34.045226,-118.238560,"{'type': 'Point', 'coordinates': [-118.2385599...",POINT (-118.23856 34.04523)
4,Modern Times Beer: The Dankness Dojo,34.044133,-118.257600,"{'type': 'Point', 'coordinates': [-118.2575999...",POINT (-118.25760 34.04413)
...,...,...,...,...,...
95,The Vegan Indian,34.091187,-118.328514,"{'type': 'Point', 'coordinates': [-118.3285140...",POINT (-118.32851 34.09119)
96,The Blue Rose,34.094118,-118.327508,"{'type': 'Point', 'coordinates': [-118.327508,...",POINT (-118.32751 34.09412)
97,Brothers Meatballs,34.097830,-118.325810,"{'type': 'Point', 'coordinates': [-118.32581, ...",POINT (-118.32581 34.09783)
98,Golden Bridge Yoga,34.096240,-118.327560,"{'type': 'Point', 'coordinates': [-118.3275604...",POINT (-118.32756 34.09624)


In [35]:
a = 0
b = float('inf')
for x in list(LA.find()):
    query2 = [{
            '$geoNear': {'near': [x['longitud'],x['latitud']],
                'distanceField': 'distancia',
                'maxDistance': 10000,
                'query': { 'nombre': {'$regex':'Starbucks' }},
                'distanceMultiplier': 6371.008,
                #'includeLocs': "dist.location",
                'spherical': True,
                }
            }]
    test2 = pd.DataFrame(LA.aggregate(query2))
    a = test2.distancia.mean()
    if a < b :
        b = a
        print(x,b)
    

{'_id': ObjectId('613227eafe06aad84d5bc160'), 'nombre': 'Starbucks', 'latitud': 34.05877990357595, 'longitud': -118.24024876785406, 'location': {'type': 'Point', 'coordinates': [-118.24024876785406, 34.05877990357595]}} 5.3753419512249865
{'_id': ObjectId('613227eafe06aad84d5bc161'), 'nombre': 'Starbucks', 'latitud': 34.049518247264665, 'longitud': -118.24190789611599, 'location': {'type': 'Point', 'coordinates': [-118.24190789611599, 34.049518247264665]}} 5.252658223413221
{'_id': ObjectId('613227eafe06aad84d5bc163'), 'nombre': 'Starbucks', 'latitud': 34.04816305210899, 'longitud': -118.25852634103549, 'location': {'type': 'Point', 'coordinates': [-118.25852634103549, 34.04816305210899]}} 4.795218258392294
{'_id': ObjectId('613227eafe06aad84d5bc164'), 'nombre': 'Starbucks', 'latitud': 34.05208, 'longitud': -118.26327, 'location': {'type': 'Point', 'coordinates': [-118.26327, 34.05208]}} 4.777232606609069
{'_id': ObjectId('613227eafe06aad84d5bc19c'), 'nombre': 'Starbucks', 'latitud': 3

In [36]:
list(LA.find())

[{'_id': ObjectId('613227eafe06aad84d5bc160'),
  'nombre': 'Starbucks',
  'latitud': 34.05877990357595,
  'longitud': -118.24024876785406,
  'location': {'type': 'Point',
   'coordinates': [-118.24024876785406, 34.05877990357595]}},
 {'_id': ObjectId('613227eafe06aad84d5bc161'),
  'nombre': 'Starbucks',
  'latitud': 34.049518247264665,
  'longitud': -118.24190789611599,
  'location': {'type': 'Point',
   'coordinates': [-118.24190789611599, 34.049518247264665]}},
 {'_id': ObjectId('613227eafe06aad84d5bc162'),
  'nombre': 'Starbucks Reserve',
  'latitud': 34.07718627690432,
  'longitud': -118.26489718236814,
  'location': {'type': 'Point',
   'coordinates': [-118.26489718236814, 34.07718627690432]}},
 {'_id': ObjectId('613227eafe06aad84d5bc163'),
  'nombre': 'Starbucks',
  'latitud': 34.04816305210899,
  'longitud': -118.25852634103549,
  'location': {'type': 'Point',
   'coordinates': [-118.25852634103549, 34.04816305210899]}},
 {'_id': ObjectId('613227eafe06aad84d5bc164'),
  'nombre':

In [54]:
Map(layers = [
    Layer(gdf, "color:green", popup_hover=[popup_element("nombre","Starbucks")]),
    Layer(gdf2, "color:red", popup_hover=[popup_element("nombre","Vegano")])
    ],
   )

In [4]:
starbucks = sr.datamatico("starbucks",la_lat_lon)

In [18]:
distancia = sr.distancia('LA',la_lat_lon,'Starbucks')

In [19]:
distancia

{'media': 5.159825153933018, 'minima': 0.1752575144116722}

In [3]:
vegano = sr.datamatico("vegan restaurant",la_lat_lon)
vegano

,type,name.vegan restaurant,latitud,longitud,location,geometry
0,vegan restaurant,Cafe Gratitude,34.045629,-118.232559,"{'type': 'Point', 'coordinates': [-118.2325587...",POINT (-118.23256 34.04563)
1,vegan restaurant,Au Lac,34.056475,-118.250488,"{'type': 'Point', 'coordinates': [-118.2504878...",POINT (-118.25049 34.05647)
2,vegan restaurant,Veggie Grill,34.048655,-118.254717,"{'type': 'Point', 'coordinates': [-118.2547171...",POINT (-118.25472 34.04865)
3,vegan restaurant,Shojin,34.045226,-118.238560,"{'type': 'Point', 'coordinates': [-118.2385599...",POINT (-118.23856 34.04523)
4,vegan restaurant,Modern Times Beer: The Dankness Dojo,34.044133,-118.257600,"{'type': 'Point', 'coordinates': [-118.2575999...",POINT (-118.25760 34.04413)
...,...,...,...,...,...,...
95,vegan restaurant,The Vegan Indian,34.091187,-118.328514,"{'type': 'Point', 'coordinates': [-118.3285140...",POINT (-118.32851 34.09119)
96,vegan restaurant,The Blue Rose,34.094118,-118.327508,"{'type': 'Point', 'coordinates': [-118.327508,...",POINT (-118.32751 34.09412)
97,vegan restaurant,Brothers Meatballs,34.097830,-118.325810,"{'type': 'Point', 'coordinates': [-118.32581, ...",POINT (-118.32581 34.09783)
98,vegan restaurant,Golden Bridge Yoga,34.096240,-118.327560,"{'type': 'Point', 'coordinates': [-118.3275604...",POINT (-118.32756 34.09624)


In [17]:
Map(layers = [
    Layer(starbucks, "color:green", popup_hover=[popup_element("name.starbucks","Starbucks")]),
    Layer(vegano, "color:red", popup_hover=[popup_element("name.vegan restaurant","Vegano")])
    ],
   )

In [4]:
m = folium.Map(location=la_lat_lon, zoom_start=12)

In [11]:
def add_layer(nombre,colorin,m,df):
    layer = folium.FeatureGroup(name=f'{nombre}')
    for i, row in df.iterrows():
        layer.add_child(
            folium.Circle(
            radius=50,
            location=[row["latitud"], row["longitud"]],
            tooltip=f"<b>{nombre}: <br></b>"+row[f"name.{nombre}"],
            color=f"{colorin}",
            fill=True
            )
        )
    layer.add_to(m)

In [9]:
add_layer('vegan restaurant','blue',m)

In [10]:
add_layer('starbucks','red',m)

KeyError: 'name.starbucks'

In [7]:
folium.LayerControl(collapsed=False).add_to(m)

In [8]:
m

In [ ]:
folium.LayerControl(collapsed=False).add_to(map_3)

In [31]:
def puntaje(colect,coord):
    client = MongoClient("localhost:27017")
    db = client.get_database("new_office")
    colec = db.get_collection(f"{colect}")
    lista = []
    for venue in colec.distinct('type'):
        lista.append(sr.distancia(colec,coord,venue).minima)
    return lista

In [17]:
 resultado = 100-((distancia(ciudad*0.2/5000)*100)

NameError: name 'LA' is not defined

In [32]:
puntaje('LA',la_lat_lon)

OperationFailure: $geoNear requires a geo index to run, but new_office.Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'new_office'), 'LA') does not exist, full error: {'ok': 0.0, 'errmsg': "$geoNear requires a geo index to run, but new_office.Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'new_office'), 'LA') does not exist", 'code': 26, 'codeName': 'NamespaceNotFound'}

In [33]:
client = MongoClient("localhost:27017")
db = client.get_database("new_office")
colec = db.get_collection(f"LA")


In [38]:
colec.distinct('type')

['Starbucks', 'Vegan restaurant']

In [36]:
sr.distancia('LA',la_lat_lon,'Starbucks')

{'media': 5.159825153933018, 'minima': 0.1752575144116722}